In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Step 1: Read and preprocess the data
data = pd.read_csv('food_data.csv')

# Handling missing values
data.fillna('', inplace=True)

# Step 2: Feature Extraction
# Combine description and food type to create input for TF-IDF
data['Features'] = data['Name'] + ' ' + data['Food_Type'] + ' ' + data['Describe']

# Convert text data (name + food type + description) into numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Features'])

# Step 3: Similarity Calculation
# Compute cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Step 4: Recommendation Generation with Calorie Constraint
def get_recommendations(query, max_calories, cosine_sim=cosine_sim):
    # Calculate the similarity scores for each food item based on name, food type, and description
    scores = {}
    for index, row in data.iterrows():
        score = 0
        if query.lower() in row['Name'].lower():
            score += 1.5
        if query.lower() in row['Food_Type'].lower():
            score += 1
        if query.lower() in row['Describe'].lower():
            score += 1
        scores[index] = score
    
    # Sort the food items based on the calculated scores
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Get the top 10 food items within the calorie limit
    recommended_foods = []
    for index, score in sorted_scores:
        if data['Calories'].iloc[index] <= max_calories:
            recommended_foods.append(data['Name'].iloc[index])
            if len(recommended_foods) == 10:
                break
    
    return recommended_foods

# Example usage
query = "vegan"
max_calories = 300
recommendations = get_recommendations(query, max_calories)
print(recommendations)




['christmas tree pizza', 'summer squash salad', 'sweet chilli almonds', 'tricolour salad', 'japanese curry arancini with barley salsa', 'chocolate nero cookies', 'lamb and chargrilled bell pepper soup', 'cream of almond soup', 'broccoli and almond soup', 'lemon honey glazed sous vide corn on the cob']
